# INF367 Mandatory 2

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from qiskit.circuit.library import UnitaryGate
from collections import Counter, defaultdict
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from matplotlib import pyplot as plt
from qiskit.result import marginal_counts
from src import Model1, Model2, Model3, circuit1, circuit2, circuit3

In [ ]:
SEED = 367

## Data exploration and pre-processing

In [ ]:
X, y = load_iris(return_X_y=True)
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=.6, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, train_size=.5, random_state=SEED)
print("Training size: ", len(X_train))
print("Validation size: ", len(X_val))
print("Test size: ", len(X_test))

In [ ]:
print("Features shape: ",X_train.shape)
print("Target shape: ",y_train.shape)
print(f"Feature value range: {np.min(X_train)} : {np.max(X_train)}")
print("Target values: ", Counter(y_train))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)

## QNN-circuits

In [ ]:
# Qircuit 1
circ1 = circuit1([1.61567622, 1.83259571, 2.34288266, 3.00500167], [1,2,1,2,1,2,1,2,3])
circ1.draw(reverse_bits=True, output="mpl")

In [ ]:
# Circuit 2
    
layers = 2
feature_size = 4
parameters = np.random.uniform(low=0, high=np.pi, size=(layers*feature_size,))
circ2 = circuit2([0, 1.64, 2.24, 3.0], parameters, layers)
circ2.draw("mpl", reverse_bits=True)

In [ ]:
# Circuit 3

layers = 2
feature_size = 4
parameters = np.random.uniform(low=0, high=2*np.pi, size=(2*layers * feature_size))
circ = circuit3([1.2, 2.1, 0.5, 4], parameters, layers)
circ.draw("mpl", reverse_bits=True)

## Training and Validation

In [ ]:
params = [
    {"learning_rate": 0.1, "epsilon": 1, "layers": 2}
]

In [ ]:
# Model 2 parameters 



In [ ]:
# epochs = 2
# for param in params:
test_model = Model2(layers=4, learning_rate=0.6, epsilon=.2, gradient_shots=1000, seed=SEED)
      
trained_model = test_model.fit(5, X_train, y_train, X_val, y_val)

In [ ]:
def plot_loss(train_loss, val_loss):
    plt.title("Training and Validation loss")
    plt.plot(train_loss, label="Training")
    plt.plot(val_loss, label="Validation")
    plt.legend()
    plt.show()

In [ ]:
plot_loss(trained_model.train_loss, trained_model.val_loss)

## Test Performance

In [ ]:
preds = trained_model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, preds))

In [ ]:
cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()